In [1]:
#!pip install git+"https://github.com/russelljjarvis/textstat.git"


In [2]:
import pickle

#TkAgg
import matplotlib
#matplotlib.use('Qt5Agg')
matplotlib.use('agg')
import matplotlib as mpl
import matplotlib.pyplot as plt


#matplotlib.use('QT4Agg')
#!pip install wxPython
#!pip install backend_wx

#WX
#QTAgg
#QT4Agg

try: 
    with open('unraveled_links.p','rb') as handle:
        unravel = pickle.load(handle)

except:
    import dask.bag as db
    from t_analysis_flat import web_iter, map_wrapper
    grid = {}
    grid['b']=[0,1,2,3]
    query_list = ['GMO','Genetically_Modified_Organism','Transgenic','Vaccine']
    grid['search_term'] = [ (i, q) for i,q in enumerate(query_list) ]
    from sklearn.grid_search import ParameterGrid
    grid = list(ParameterGrid(grid))
    grid = [(dicti['search_term'][0],dicti['search_term'][1],dicti['b']) for dicti in grid ]
    #from dask.distributed import Client
    import dask.bag as db
    grid = db.from_sequence(grid,npartitions = 8)
    list_per_links = list(db.map(web_iter,grid).compute())
    
    remove_empty = [i for i in list_per_links if len(i)>0 ]
    unravel = []
    for i in remove_empty:
        unravel.extend(i)
    
    with open('unraveled_links.p','wb') as handle:
        pickle.dump(unravel,handle)
    #with open('unraveled_links.p','rb') as handle:
    #    assert handle
    #    unravel = pickle.load(handle)
        

import pandas as pd


In [3]:
unravel[0]['urlDat']

{'cliau': 24.17,
 'dcr': 8.56,
 'dw': 1382,
 'fkg': 12.9,
 'fre': 24.85,
 'gf': 17.930464933018126,
 'keyword': 'GMO',
 'link_rank': 0,
 'lwf': 4.583333333333333,
 'ri': 19.2,
 'se': 'google_',
 'sentcount': 188,
 'smog': 10.3,
 'sp': 0.055084360084360046,
 'ss': 0.4824196482529815,
 'standard': '12th and 13th grade',
 'stfreq': 161,
 'wcount': 5076}

In [4]:
#print(list_per_links[1])\

bings = [ f for f in unravel if f['urlDat']['se'] =='bing_']
#print(bings)
googles = [ f for f in unravel if f['urlDat']['se'] =='google_']
GMOs = [ f for f in unravel if f['urlDat']['keyword'] =='GMO']
GMO_standard = [ f['urlDat']['standard'] for f in unravel if f['urlDat']['keyword'] =='GMO' ]
GMO_rank = [ f['urlDat']['link_rank'] for f in unravel if f['urlDat']['keyword'] =='GMO' ]

GMO_subjectivity_pol = [ f['urlDat']['sp'] for f in unravel if f['urlDat']['keyword'] =='GMO' ]
GMO_subjectivity_sent = [ f['urlDat']['ss'] for f in unravel if f['urlDat']['keyword'] =='GMO' ]

GMO_rank

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [5]:


import seaborn as sns

se = {}
se[0] = 'google_'
se[1] = 'gScholar_'
se[2] = 'bing_'
se[3] = 'yahoo_'

ses = list(se.values())
search_query = str('Vaccine')
searchList = ['GMO','Genetically_Modified_Organism','Transgenic','Vaccine']

for outer,search_query in enumerate(searchList):    
    x = []
    y = []
    for i,v in enumerate(ses):
        print(i,v)
        complexity = [ f['urlDat']['fkg'] for f in unravel if f['urlDat']['keyword'] ==search_query and f['urlDat']['se'] == v ]
        rank = [ f['urlDat']['link_rank'] for f in unravel if f['urlDat']['keyword'] ==search_query and f['urlDat']['se'] == v ]
        #print(complexity[0][0],complexity[0][0])
        x.append(rank)
        y.append(complexity)
    print(len(complexity),len(rank))
    fig, ax = plt.subplots()
    plt.title(str('reading grade ')+search_query)
    plt.ylabel('text reading level')
    plt.xlabel('link rank')
    plt.scatter(x[0],y[0],label=ses[0])
    plt.scatter(x[1],y[1],label=ses[1])
    plt.scatter(x[2],y[2],label=ses[2])
    plt.scatter(x[3],y[3],label=ses[3])
    
    #labels = [ses[0],ses[1],ses[2],ses[3]]
    #lineObjects = plt.plot(x[0],y[0],'r',x[1],y[1],'g',x[2],y[2],'b',x[3],y[3],'k')
    #plt.legend(lineObjects,labels)
    legend = ax.legend(loc='upper center', shadow=True)
    plt.savefig('rank_versus_complexity_{0}.png'.format(search_query))
    plt.clf()
import textstat
import numpy as np




0 google_
1 gScholar_
2 bing_
3 yahoo_
0 0
0 google_
1 gScholar_
2 bing_
3 yahoo_
0 0
0 google_
1 gScholar_
2 bing_
3 yahoo_
0 0
0 google_
1 gScholar_
2 bing_
3 yahoo_
0 0


In [6]:
# 2.       Pro/anti/neutral vs. text complexity#

for outer,search_query in enumerate(searchList):    
    x = []
    y = []
    for i,v in enumerate(ses):
        #print(i,v)
        complexity = [ f['urlDat']['fkg'] for f in unravel if f['urlDat']['keyword'] ==search_query and f['urlDat']['se'] == v ]
        sent = [ f['urlDat']['sp'] for f in unravel if f['urlDat']['keyword'] ==search_query and f['urlDat']['se'] == v ]
        #print(complexity[0][0],complexity[0][0])
        x.append(sent)
        y.append(complexity)

    fig, ax = plt.subplots()
    plt.title(str('sentiment ')+search_query)
    plt.ylabel('text reading level')
    plt.xlabel('sentiment polarity')
    labels = [ses[0],ses[1],ses[2],ses[3]]
    #lineObjects = 
    plt.scatter(x[0],y[0],label=ses[0])
    plt.scatter(x[1],y[1],label=ses[1])
    plt.scatter(x[2],y[2],label=ses[2])
    plt.scatter(x[3],y[3],label=ses[3])

    legend = ax.legend(loc='upper center', shadow=True)
    plt.savefig('sentiment_vs_complexity{0}.png'.format(search_query))
    plt.clf()
import textstat
import numpy as np
#




#:

In [7]:
# 3.       GMO/transgenics vs. text complexity

import seaborn as sns
se = {}
se[0] = 'google_'
se[1] = 'gScholar_'
se[2] = 'bing_'
se[3] = 'yahoo_'

ses = list(se.values())
search_query = str('Vaccine')
search_query = ['GMO','Transgenic']
xx=[]
yy=[]
for outer,sq in enumerate(search_query):    
    x = []
    y = []
    for i,v in enumerate(ses):
        complexity = [ f['urlDat']['fkg'] for f in unravel if f['urlDat']['keyword'] ==sq and f['urlDat']['se'] == v and f['urlDat']['link_rank'] == 0 ]
        if len(complexity)>0:
            comp = float(complexity[0])
        else:
            comp = 0
        xx.append(i)
        y.append(comp)    
    yy.append(y)
print(len(complexity),len(rank))
fig, ax = plt.subplots()
# Two subplots, the axes array is 1-d
width = 0.35       # the width of the bars
plt.clf()
fig, ax = plt.subplots()

#f, axarr = plt.subplots(1, sharex=True)
ind = [i for i,v in enumerate(ses)]
offset = [(i+width)/2.0 for i,v in enumerate(ses)]

rects1 = ax.bar(ind,yy[0],width, color='r')# yerr=men_std)
rects2 = ax.bar(ind, yy[1], width, color='y')#, yerr=women_std)
# add some text for labels, title and axes ticks
ax.set_ylabel('Complexity')
ax.set_title('Complexity by search query and Search Engine')
ax.set_xticks(offset)
ax.set_xticklabels((se[0], se[1], se[2], se[3]))#, 'G5'))

ax.legend((rects1[0], rects2[0]), ('GMO', 'Transgenic'))

plt.savefig('GMO_Transgenic_complexity_{0}{1}.png'.format(search_query[0],search_query[1]))
import numpy as np


0 0


In [19]:

import seaborn as sns
se = {}
se[0] = 'google_'
se[1] = 'gScholar_'
se[2] = 'bing_'
se[3] = 'yahoo_'

ses = list(se.values())
search_query = str('Vaccine')
search_query = ['GMO','Transgenic']
xx=[]
yy=[]
for outer,sq in enumerate(search_query):    
    x = []
    y = []
    for i,v in enumerate(ses):
        complexity = [ f['urlDat']['fkg'] for f in unravel if f['urlDat']['keyword'] ==sq and f['urlDat']['se'] == v and f['urlDat']['link_rank'] == 0 ]
        #complexity = [ f['urlDat']['fkg'] for f in unravel if f['urlDat']['keyword'] ==sq and f['urlDat']['se'] == v and f['urlDat']['link_rank'] == 0 ]
        sent = [ f['urlDat']['sp'] for f in unravel if f['urlDat']['keyword'] == sq and f['urlDat']['se'] == v and f['urlDat']['link_rank'] == 0 ]
        if len(complexity)>0:
            comp = float(complexity[0])
        else:
            comp = 0
        if len(sent)>0:
            sent = float(sent[0])
        else:
            sent = 0
    
        xx.append(i)
        y.append((comp,sent))    
    yy.append(y)
print(len(complexity),len(rank))
fig, ax = plt.subplots()
# Two subplots, the axes array is 1-d
width = 0.35       # the width of the bars
plt.clf()
fig, ax = plt.subplots()

#f, axarr = plt.subplots(1, sharex=True)
ind = [i for i,v in enumerate(ses)]
offset = [(i+width)/2.0 for i,v in enumerate(ses)]



fig, ax = plt.subplots()
plt.title(str('sentiment '))#+search_query)
plt.ylabel('text reading level')
plt.xlabel('sentiment polarity')
#lineObjects = 
sentt0 = [sent[1] for sent in yy[0] ]
complexity0 = [sent[0] for sent in yy[0] ]
plt.scatter(sentt0,complexity0,label='GMO')
sentt1 = [sent[1] for sent in yy[1] ]
complexity1 = [sent[0] for sent in yy[0] ]
plt.scatter(sentt1,complexity1,label='Transgenic')


#    plt.scatter(x[0],y[0],label=ses[0])
#    plt.scatter(x[1],y[1],label=ses[1])

legend = ax.legend(loc='upper center', shadow=True)
#ax.set_xticks(offset)
#ax.set_xticklabels((se[0], se[1], se[2], se[3]))#, 'G5'))

#plt.savefig('sentiment_vs_complexity{0}.png'.format(search_query))
#plt.clf()


plt.savefig('Figure2{0}{1}.png'.format(search_query[0],search_query[1]))


0 0


/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
